# sklearn の iris データを読み込む
* 機械学習の基本のキ → 教師データを学習用と、検証用に分ける

In [1]:
import numpy as np
from sklearn import datasets
# データセットのロード
# iris.data = [(がく片の長さ , がく片の幅 , 花びらの長さ , 花びらの幅)]
iris = datasets.load_iris()
x_vals = np.array([x[0:3] for x in iris.data])
y_vals = np.array([x[3] for x in iris.data])

data_size = len(x_vals)
train_size = int(data_size * 0.8)
test_size = data_size - train_size

x_train = x_vals[0:train_size]
y_train = y_vals[0:train_size]

x_test = x_vals[train_size:data_size]
y_test = y_vals[train_size:data_size]

print ("total:{} = train:{} + test:{}".format(data_size, len(x_train), len(x_test)))


total:150 = train:120 + test:30


# 共通関数の準備

In [2]:
isDebugEnabled = True
def debug(msg, *args) :
    if isDebugEnabled :
        print(msg.format(*args))

def sigmoid(x) :
    """
    Sigmoid (シグモイド関数)
    @param x
    @return sigmoid(x)
    """
    return 1.0 / (1.0 + np.exp(-1.0 * x))

def relu(x) :
    """
     Rectified Linear Unit (正規化線形関数).
    @param x
    @return max(0,x)
    """
    return np.maximum(0, x)

def identity_mapping(x) :
    """
    恒等写像です.
    @param x 
    @return xをそのまま返します
    """
    return x

def d_sigmoid(x):
    """
    Sigmoid (シグモイド関数)の導関数
    @param x
    @return sigmoid'(x)
    """
    dx = (1.0 - sigmoid(x)) * sigmoid(x)
    return dx

def d_relu(x):
    """
     ReLuの導関数.
    @param x
    @return relu'(x)
    """
    return np.where( x > 0, 1, 0)

def least_square(d, y):
    """
    自乗誤差を求めます.
    @param d 教師データ(expected)
    @param y 予想(actual)
    """
    return np.sum(np.square(d - y)) / 2

# ニューラルネットワーク作成関数

In [3]:
def create_layer(in_size, out_size) :
    """
    1レイヤ分をランダムに初期化します.
    @param in_size 入力サイズ
    @param out_size 出力サイズ
    @return w 重み行列
    @return b バイアス
    """
    w = np.random.rand(out_size, in_size)
    b = np.random.rand(out_size)
    return w,b

def create_network(*units) :
    """
    nレイヤ分のネットワークを作成します
    @param *units 中間層のサイズを可変引数で指定します
    @return w 重み行列
    @return b バイアス
    """
    w_lst = []
    b_lst = []
    for layer in range(0, len(units) - 1) :
        in_size = units[layer]
        out_size = units[layer+1]
        w,b = create_layer(in_size, out_size)
        w_lst.append(w)
        b_lst.append(b)
    return w_lst, b_lst

# 順伝搬

In [6]:
def forward(x,w,b,func) :
    """
    順伝搬します.
    
    z(0) = x
    for l = 1 to L
      u(l) = W(l) ・ z(l-1) + b(l)
      z(l) = relu( u(l) )
    y = func( z(L) )
    
    @param x 入力データ　(複数のデータを同時に投入できる)
    @param w 重み行列
    @param b バイアス
    @param func 出力層の活性化関数
    @return y 出力
    @return u 中間層の計算結果
    """
    #　投入されたミニバッチの大きさを取得
    batch_size = 1 if 1 == len(x.shape) else x.shape[1] 
    debug("batch size = {}", batch_size)
    
    z = x
    u_lst = []
    for layer in range(0, len(w)) :
        debug("layer [{}] w = \n{}", layer, w[layer])
        debug("layer [{}] z = {}", layer, z)
        debug("layer [{}] b = {}", layer, b[layer])
        
        # ミニバッチで、バイアスを計算するために、各行がバイアスで、ミニバッチの大きさ
        # 分だけ列数がある bias 　行列を作る。作り方は、
        # 対角成分がバイアスな行列 ・ 行数バイアスの大きさ ×列数ミニバッチの大きさで成分が1な行列 =
        #  [
        #   [b1 b1 b1 ... b1]
        #   [b2 b2 b2 ... b2]
        #   [b3 b3 b3 ... b3]
        # ]
        bias_size = b[layer].shape[0]
        bias = (np.identity(bias_size) * b[layer]).dot( np.ones((bias_size, batch_size)))
        
        u = w[layer].dot(z) + bias
        u_lst.append(u)
        z = func(u)
        debug("layer [{}] output = {}", layer, z)
    # 出力は恒等写像とする
    y = identity_mapping(z)
    return y, u_lst

# 確認用ネットワーク作成
w,b = create_network(3,3,1)

# 順伝搬の動作確認1 (1データの投入)
print("----------")
# 0 番目のデータを初期ネットワークで順伝搬してみます
# 出力層の活性化関数は 恒等写像 (identity_mapping)
y, u = forward(x_train[0], w, b, identity_mapping)
print("D (expect) : {}, Y (actual) : {}, E (error): {}".format(y_train[0], y, least_square(y_train[0], y)))
print("U (mid layers) : {}".format(u))

# 順伝搬の動作確認2 (データのバッチ投入)
print("----------")
isDebugEnabled = False
y,u = forward(x_train.T, w, b, identity_mapping)
print("E (error): {}".format(least_square(y_train, y)))


----------
D (expect) : 0.2, Y (actual) : [[13.04852271  8.67349662 13.87669814]], E (error): 211.96837640412718
U (mid layers) : [array([[6.60820352, 4.21302467, 7.06160156],
       [6.79506478, 4.39988592, 7.24846282],
       [6.83341618, 4.43823733, 7.28681422]]), array([[13.04852271,  8.67349662, 13.87669814]])]
----------
E (error): 8390.597265775443


# 逆伝搬

In [5]:
def backword(w,b,u,y,d,func) :
    """
    逆伝搬します.
    @param w 重み行列
    @param b バイアス
    @return u 中間層の計算結果
    @return y 出力
    @return d 教師値
    @param func 出力層の活性化関数の導関数
    @return w 更新後の重み行列
    @return b 更新後のバイアス
    """
    # TODO Implement me!!